In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import numpy as np

# If you keep separate dirs:
MODEL_DIR = "./distilbert-imdb-final"
TOKENIZER_DIR = "./my_finetuned_model"   # or use MODEL_DIR if you re-save there

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

raw = load_dataset("imdb")
test = raw["test"]  # 25k examples

In [ ]:
model_name = "./finetuned_distilbert_imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [3]:
def predict_batch(texts, batch_size=32):
    preds = []
    for i in range(0, len(texts), batch_size):
        batch = texts[ i : i+batch_size ]
        enc = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**enc).logits
        preds.extend(torch.argmax(logits, dim=-1).cpu().numpy().tolist())
    return np.array(preds)

def accuracy(labels, preds):
    return (labels == preds).mean()

In [4]:
subset = test.select(range(5000))  # use a subset for speed; increase if you like
texts = subset["text"]
labels = np.array(subset["label"])

baseline_preds = predict_batch(texts)
baseline_acc = accuracy(labels, baseline_preds)
print({"baseline_acc": float(baseline_acc)})

{'baseline_acc': 1.0}
